In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.axes_grid1 import make_axes_locatable
from rtree import index
from datetime import datetime, timedelta
import contextily as ctx
import folium
from collections import defaultdict
import os

In [2]:
from shapely.geometry import Point
from shapely.ops import transform
import pyproj
from functools import partial
import logging
from tqdm import tqdm

In [3]:
# Set path
path = r'/media/scruffy/Elements/python_data/Final'

In [137]:
# Load climate variables as DataFrames
df2015 = pd.read_csv(os.path.join(path, 'era5_2015.csv'))
df2016 = pd.read_csv(os.path.join(path, 'era5_2016.csv'))
df2017 = pd.read_csv(os.path.join(path, 'era5_2017.csv'))
df2018 = pd.read_csv(os.path.join(path, 'era5_2018.csv'))
df2019 = pd.read_csv(os.path.join(path, 'era5_2019.csv'))
df2020 = pd.read_csv(os.path.join(path, 'era5_2020.csv'))
df2021 = pd.read_csv(os.path.join(path, 'era5_2021.csv'))

In [157]:
def convert_to_geodataframes(dataframes_dict):

    geo_dataframes = {}
    
    for name, df in dataframes_dict.items():
        # Create point geometries
        geometry = gpd.points_from_xy(df['longitude'], df['latitude'])
        
        # Create GeoDataFrame
        gdf = gpd.GeoDataFrame(
            df, 
            geometry=geometry, 
            crs="EPSG:4326"
        )
        # Convert CRS to UTM for accurate distance
        utm_crs = gdf.estimate_utm_crs()
        gdf = gdf.to_crs(utm_crs).copy()
        geo_dataframes[name] = gdf
        print(f"Converted {name} to GeoDataFrame with {len(gdf)} points")
    
    return geo_dataframes


In [158]:
dataframes = {
    'gdf2015': df2015,
    'gdf2016': df2016,
    'gdf2017': df2017,
    'gdf2018': df2018,
    'gdf2019': df2019,
    'gdf2020': df2020,
    'gdf2021': df2021
}

# Convert all to GeoDataFrames
geo_dataframes = convert_to_geodataframes(dataframes)

Converted gdf2015 to GeoDataFrame with 1237600 points
Converted gdf2016 to GeoDataFrame with 1234221 points
Converted gdf2017 to GeoDataFrame with 1227495 points
Converted gdf2018 to GeoDataFrame with 1227495 points
Converted gdf2019 to GeoDataFrame with 1227495 points
Converted gdf2020 to GeoDataFrame with 1230858 points
Converted gdf2021 to GeoDataFrame with 1227495 points


In [171]:
gdf2016 = geo_dataframes['gdf2016']
gdf2017 = geo_dataframes['gdf2017']
gdf2018 = geo_dataframes['gdf2018']
gdf2019 = geo_dataframes['gdf2019']
gdf2020 = geo_dataframes['gdf2020']
gdf2021 = geo_dataframes['gdf2021']

In [57]:
fire_gdf = gpd.read_file(os.path.join(path, 'fires_clean.geojson'))

In [6]:
fires_since_2005 = fire_gdf[fire_gdf['ALARM_DATE']>='2005-01-01']

In [7]:
gdf2 = fires_since_2005.copy()

In [ ]:
# Convert dates to datetime
gdf1['date_dt'] = pd.to_datetime(gdf1['date'])
gdf2['date_dt'] = pd.to_datetime(gdf2['ALARM_DATE'])

In [ ]:
# Create buffered geometries (8000m buffer)
gdf1['buffered_geom'] = gdf1.geometry.buffer(8000)

In [179]:
def wildfire_analysis(gdf1, gdf2, lat_col='latitude', lon_col='longitude'):
    """
    Wildfire historical analysis with:
    1. Pre-filtering of fire data by date range
    2. Vectorized operations wherever possible
    3. Parallel processing of coordinate groups
    
    Args:
        gdf1: Primary GeoDataFrame (climate data with point coordinates)
        gdf2: Secondary GeoDataFrame (fire data)
        lat_col: Latitude column in climate_gdf
        lon_col: Longitude column in climate_gdf
        
    Returns:
        GeoDataFrame with four new columns:
        - active_fire (bool)
        - active_fire_acres (float)
        - fire_past_5yrs (bool)
        - fires_past_10yrs_count (int)
    
    """
    
    # Build spatial index for fire data
    print("Building spatial index...")
    fire_idx = index.Index()
    fire_records = []  # Store fire records with their bounds and dates
    
    # Pre-compute bounds and store fire records
    for i, (_, row) in enumerate(gdf2.iterrows()):
        bounds = row.geometry.bounds
        fire_idx.insert(i, bounds)
        fire_records.append({
            'bounds': bounds,
            'geometry': row.geometry,
            'acres': row['GIS_ACRES']
        })
    
    # Initialize result columns with desired data types
    gdf1['active_fire'] = False
    gdf1['active_fire_acres'] = 0.0
    gdf1['fire_past_5yrs'] = False
    gdf1['fires_past_10yrs_count'] = 0
    
    # Group by coordinates and process in parallel
    print("Processing coordinate groups...")
    coord_groups = gdf1.groupby([lat_col, lon_col])
    
    # Process each unique location
    for (lat, lon), group_df in tqdm(coord_groups, desc="Processing locations"):
        # Get representative buffered geometry for this location
        buffered_geom = group_df['buffered_geom'].iloc[0]
        
        # Find potentially intersecting fires using spatial index
        possible_matches = list(fire_idx.intersection(buffered_geom.bounds))
        if not possible_matches:
            continue
            
        # Process each date at this location
        for _, row in group_df.iterrows():
            current_date = row['date_dt']
            date_5yr = current_date - timedelta(days=5*365)
            date_10yr = current_date - timedelta(days=10*365)
            
            # Initialize counters
            active_fire = False
            active_acres = 0.0
            fire_5yr = False
            fire_count_10yr = 0
            
            # Check each potential fire match
            for match_idx in possible_matches:
                fire_record = fire_records[match_idx]
                
                # Skip if fire date is outside our date range
                if fire_record['date'] > current_date or fire_record['date'] < date_10yr:
                    continue
                    
                # Check spatial intersection
                if not buffered_geom.intersects(fire_record['geometry']):
                    continue
                    
                # Check for active fire (same date)
                if fire_record['date'] == current_date:
                    active_fire = True
                    active_acres = max(active_acres, fire_record['acres'])
                
                # Count for 10-year window
                fire_count_10yr += 1
                
                # Check for 5-year window
                if fire_record['date'] >= date_5yr:
                    fire_5yr = True
            
            # Update results
            gdf1.loc[row.name, 'active_fire'] = active_fire
            gdf1.loc[row.name, 'active_fire_acres'] = active_acres
            gdf1.loc[row.name, 'fire_past_5yrs'] = fire_5yr
            gdf1.loc[row.name, 'fires_past_10yrs_count'] = fire_count_10yr
    
    # Clean up temporary columns
    # gdf1.drop(columns=['buffered_geom'], inplace=True)
    
    return gdf1

In [180]:
gdf_16_proc = wildfire_analysis(gdf1=gdf2016, gdf2=gdf2, lat_col='latitude', lon_col='longitude')

Building optimized spatial index...
Processing coordinate groups...


Processing locations: 100%|█████████████████| 3363/3363 [28:48<00:00,  1.95it/s]


In [187]:
gdf_21_proc = optimized_wildfire_analysis(gdf1=gdf2021, gdf2=gdf2, lat_col='latitude', lon_col='longitude')
gdf_21_csv = gdf_21_proc.drop(columns=['geometry'])
gdf_21_csv.to_csv(os.path.join(path, 'fire_features_2021.csv'))

gdf_20_proc = optimized_wildfire_analysis(gdf1=gdf2020, gdf2=gdf2, lat_col='latitude', lon_col='longitude')
gdf_20_csv = gdf_20_proc.drop(columns=['geometry'])
gdf_20_csv.to_csv(os.path.join(path, 'fire_features_2020.csv'))

gdf_19_proc = optimized_wildfire_analysis(gdf1=gdf2019, gdf2=gdf2, lat_col='latitude', lon_col='longitude')
gdf_19_csv = gdf_19_proc.drop(columns=['geometry'])
gdf_19_csv.to_csv(os.path.join(path, 'fire_features_2019.csv'))

gdf_18_proc = optimized_wildfire_analysis(gdf1=gdf2018, gdf2=gdf2, lat_col='latitude', lon_col='longitude')
gdf_18_csv = gdf_18_proc.drop(columns=['geometry'])
gdf_18_csv.to_csv(os.path.join(path, 'fire_features_2018.csv'))

gdf_17_proc = optimized_wildfire_analysis(gdf1=gdf2017, gdf2=gdf2, lat_col='latitude', lon_col='longitude')
gdf_17_csv = gdf_17_proc.drop(columns=['geometry'])
gdf_17_csv.to_csv(os.path.join(path, 'fire_features_2017.csv'))

Building optimized spatial index...
Processing coordinate groups...


Processing locations: 100%|███████████████| 3363/3363 [1:43:50<00:00,  1.85s/it]


Building optimized spatial index...
Processing coordinate groups...


Processing locations: 100%|█████████████████| 3363/3363 [56:42<00:00,  1.01s/it]


Building optimized spatial index...
Processing coordinate groups...


Processing locations: 100%|█████████████████| 3363/3363 [38:25<00:00,  1.46it/s]


Building optimized spatial index...
Processing coordinate groups...


Processing locations: 100%|█████████████████| 3363/3363 [37:39<00:00,  1.49it/s]


Building optimized spatial index...
Processing coordinate groups...


Processing locations: 100%|█████████████████| 3363/3363 [32:18<00:00,  1.73it/s]


In [181]:
gdf_csv = gdf_16_proc.drop(columns=['geometry'])
gdf_csv.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1234221 entries, 0 to 1247799
Data columns (total 21 columns):
 #   Column                             Non-Null Count    Dtype  
---  ------                             --------------    -----  
 0   date                               1234221 non-null  object 
 1   longitude                          1234221 non-null  float64
 2   latitude                           1234221 non-null  float64
 3   temperature_2m                     1234221 non-null  float64
 4   temperature_2m_max                 1234221 non-null  float64
 5   total_precipitation_sum            1234221 non-null  float64
 6   dewpoint_temperature_2m            1234221 non-null  float64
 7   u_component_of_wind_10m            1234221 non-null  float64
 8   v_component_of_wind_10m            1234221 non-null  float64
 9   volumetric_soil_water_layer_1      1234221 non-null  float64
 10  surface_net_solar_radiation_sum    1234221 non-null  float64
 11  surface_net_thermal_radiation

In [1]:
gdf_csv['active_fire'].value_counts().sort_values()

NameError: name 'gdf_csv' is not defined